<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Prokka-Comparison/blob/main/BATH_file_deduplication_(Positive_Negative)_Oct724.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strand-specific Genomic Data Deduplication Using E-value and Overlap Metrics


A method for filtering and deduplicating genomic data is designed to enhance the accuracy and reliability of sequence alignments by leveraging E-value thresholds and alignment overlap metrics. Initially, an E-value threshold of 0.000001 is applied to exclude low-confidence alignments. Subsequently, the DNA strand (positive or negative) is identified based on alignment positions. For each strand, a series of deduplication steps are performed:

*   100% Deduplication: Exact duplicates are identified and removed by comparing E-values and scores, retaining only the highest quality alignments.
*   70% Deduplication: Alignments with significant overlap (≥70%) are addressed by comparing E-values and sequence lengths, with the more reliable alignment being retained.
*  <70% Deduplication: For alignments with minor overlap (0.01% to <70%), adjustments are made to the alignment positions to resolve conflicts, ensuring non-redundancy of the retained alignments.

Each deduplication step is applied to ensure that the final dataset is both comprehensive and non-redundant, providing high-confidence data for subsequent analyses. This method is applied separately to positive and negative strands, allowing for tailored processing and accurate strand-specific results.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Input file specification

bathsearch_op_path = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/BATH_Deduplication/BATHOUTPUT/*.tbl"
deduplication_op_path = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Iteration_Sep30/dedup_combinned_binned_unbinned_sep30.xlsx"

In [3]:
e_value_threshold = 0.000001

In [4]:
import pandas as pd
from glob import glob
import os

In [5]:
class filtering_operations:

  @staticmethod
  def e_value_filtering(df):
    return df.loc[df['E-value']<=e_value_threshold]

  @staticmethod
  def pos_neg_strand_filtering(df_filtered):
    #df should be post e_value_filteration
    df = filtering_operations.e_value_filtering(df_filtered)
    #Strand identification
    df['ali from'] = df['ali from'].astype(int)
    df['ali to'] = df['ali to'].astype(int)
    strand = []

    for index,row in df.iterrows():
      if row['ali from'] < row['ali to']:
        strand.append("+")
      elif row['ali from'] > row['ali to']:
        strand.append("-")

    df['strand'] = strand
    return df

  @staticmethod
  def get_specific_strand(df_raw,strand_info):
    df = filtering_operations.pos_neg_strand_filtering(df_raw)
    return df.loc[df['strand']==strand_info]


In [6]:
class overlap_deduplications:

  ali_to_column = ""
  ali_from_column = ""

  # Initialize variables to hold counts
  initial_count = 0
  count_after_100 = 0
  count_after_70 = 0
  count_after_less_than_70 = 0

  #METHODS FOR 100% DEDUPLICATION

  @staticmethod
  def handle_group(group):
    #Now for each element in group
    min_e_value = group['E-value'].min()
    #find elements with lowest e-values
    group_min_e = group.loc[group['E-value'] == min_e_value]
    #if group contains a single row save it
    if len(group_min_e) == 1:
      return group_min_e
    #if group contains multiple rows with same e-values check min score
    elif len(group_min_e) > 1:
      print("Group contains multiple rows")
      return group_min_e.iloc[0:1]

  @staticmethod
  def deduplicate_full_overlaps(df):
    #100% Overlap Dedup
    overlap_deduplications.initial_count = df.shape[0]
    print("Number of elements in relevant strand: ",df.shape)
    #Find all the duplicates
    duplicates = df.duplicated(subset=['target name',overlap_deduplications.ali_from_column, overlap_deduplications.ali_to_column], keep=False)
    #make a duplicates df
    duplicates_df = df[duplicates]
    #make a non duplicates df
    not_duplicate_df = df[~duplicates]
    print("Number of exact duplicates or homologs: ",duplicates_df.shape)
    #Sort by E-value and score
    duplicates_df1 = duplicates_df.sort_values(by=['E-value', 'score'], ascending=[True, False])
    deduplicated_remhomologs = duplicates_df1.groupby(['target name',overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column], group_keys=False).apply(overlap_deduplications.handle_group).reset_index(drop=True)
    #rejion after deduplication
    dedup_step1 = pd.concat([not_duplicate_df,deduplicated_remhomologs]).sort_values(['target name','ali from','E-value'])
    overlap_deduplications.count_after_100 = dedup_step1.shape[0]
    print("Number of elements after removing exact duplicates or homologs: ",dedup_step1.shape)
    print("Elements removed: ",df.shape[0] - dedup_step1.shape[0])
    return dedup_step1

  #METHODS FOR 70% DEDUPLICATION

  @staticmethod
  def calculate_overlap(hit_a, hit_b):
    #print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
    start_a, end_a = hit_a[overlap_deduplications.ali_from_column], hit_a[overlap_deduplications.ali_to_column]
    start_b, end_b = hit_b[overlap_deduplications.ali_from_column], hit_b[overlap_deduplications.ali_to_column]

    overlap_length = max(0, min(end_a, end_b) - max(start_a, start_b) + 1)
    #print(min(end_a, end_b))
    #print(max(start_a, start_b))
    #print("Overlap length",overlap_length)

    length_a = end_a - start_a + 1
    length_b = end_b - start_b + 1

    #print("Length",length_a,length_b)
    if length_a > 0 and length_b > 0:
        overlap_perc_a = (overlap_length / length_a) * 100
        overlap_perc_b = (overlap_length / length_b) * 100
        return overlap_perc_a, overlap_perc_b
    else:
        return 0, 0

  @staticmethod
  def calculate_winner(hit_a, hit_b):
      overlap1, overlap2 = overlap_deduplications.calculate_overlap(hit_a, hit_b)
      print(f"Comparing:\nHit A: {hit_a}\nHit B: {hit_b}\nOverlap1: {overlap1}%, Overlap2: {overlap2}%")

      # Check for significant overlap
      if overlap1 >= 70 or overlap2 >= 70:
          # Determine which hit to retain based on E-value or sequence length on tie
          if hit_a['E-value'] < hit_b['E-value']:
              print("Choosing Hit A based on E-value")
              return hit_a
          elif hit_a['E-value'] > hit_b['E-value']:
              print("Choosing Hit B based on E-value")
              return hit_b
          else:  # E-values are tied, check the sequence length
              if hit_a['seq len'] > hit_b['seq len']:
                  print("E-values tied. Choosing Hit A based on larger sequence length")
                  return hit_a
              else:
                  print("E-values tied. Choosing Hit B based on larger sequence length")
                  return hit_b
      else:
          print("No significant overlap or both hits retained")
          return hit_a, hit_b

  @staticmethod
  def handle_70_overlaps(group):
    # Check if the DataFrame has 1 or fewer rows
    initial_group_count = len(group)
    if len(group) <= 1:
        return group

    items = []
    print("HIIII")
    print(group)

    for index, row in group.iterrows():
      row_tuple = row.to_dict()
      items.append(row_tuple)

    i = 0
    all_winners = []

    while i<len(items)-1:
      current = items[i]
      print(current)
      next_item = items[i+1]
      print(next_item)
      print("Begining Comparison: ",i)
      #print(current,"\n",next_item)
      result = overlap_deduplications.calculate_winner(current, next_item)

      #If result is two items append the current item to winners list since overlap of current item with next item is less than 70% so we want to keep both
      if isinstance(result,tuple):
        if result[0] not in all_winners:
          all_winners.append(result[0])
          #print("Tuple returned!")
          #print("Appending: ",result[0])
          #print("All winners currently: ",all_winners)
        #else:
        #  print("Result already in all winners, no appending!")
        i += 1
      else:
        if result == current:
          #print("Result is ",i," element and not ",i+1," popping: ",items[i+1])
          items.pop(i+1)
        else:
          i += 1

      if i >= len(items) - 1 and items[-1] not in all_winners:
        all_winners.append(items[-1])

      final_group_count = len(pd.DataFrame(all_winners))
      overlap_deduplications.count_after_70 += final_group_count

    return pd.DataFrame(all_winners)

  #METHODS FOR LESS THAN 70% DEDUPLICATION
  @staticmethod
  def calculate_below_70_changes(hit_a, hit_b):
    overlap1, overlap2 = overlap_deduplications.calculate_overlap(hit_a, hit_b)
    #print("Overlap 1: ",overlap1)
    if (0.01 <= overlap1 < 70) and (0.01 <= overlap2 < 70):
      if hit_a['E-value'] < hit_b['E-value']:
        print("Case A: A dominates")
        print(hit_a)
        print(hit_b)
        hit_b['ali from'] = hit_a['ali to'] + 1
        #print(hit_b)
        return hit_a, hit_b
      else:
        print("Case B: B dominates")
        print(hit_a)
        print(hit_b)
        hit_a['ali to'] = hit_b['ali from'] - 1
        #print(hit_a)
        #print(hit_b)
        return hit_a, hit_b
    else:
      return hit_a,hit_b

  @staticmethod
  def below_70_overlap(group):
    initial_group_count = len(group)
    if len(group) <= 1:
        print("Length of group less than 1.")
        return group

    items = []
    for index, row in group.iterrows():
      row_tuple = row.to_dict()
      items.append(row_tuple)

    for i in range(0,len(items)-1):
      element1, element2 = overlap_deduplications.calculate_below_70_changes(items[i],items[i+1])
      items[i] = element1
      items[i+1] = element2

    final_group_count = len(pd.DataFrame(items))
    overlap_deduplications.count_after_less_than_70 += final_group_count
    return pd.DataFrame(items)


  # FINAL METHOD TO CALL
  @staticmethod
  def choose_strand_operations(df):
    if df['strand'].values[0] == '-':
      print("Processing negative strand data...")
      #create the flip columns
      df['ali from flip'] = df['ali to']
      df['ali to flip'] = df['ali from']
      overlap_deduplications.ali_to_column = 'ali to flip'
      overlap_deduplications.ali_from_column = 'ali from flip'
      #100% dedup
      df = df.sort_values(by=['ali from flip','E-value'], ascending=[True, True])
      #print(df)
      step1_dedup = overlap_deduplications.deduplicate_full_overlaps(df)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("100% deduplication done.")
      #70% Overlap
      step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("70% deduplication done.")
      #Less than 70% overlap
      step3_dedup = step2_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.below_70_overlap).reset_index(drop=True)
      print("Less than 70% deduplication done.")
      #return something final
      return step3_dedup
    elif df['strand'].values[0] == '+':
      print("Processing positive strand data...")
      overlap_deduplications.ali_to_column = 'ali to'
      overlap_deduplications.ali_from_column = 'ali from'
      #100% dedup
      df = df.sort_values(by=['ali from','E-value'], ascending=[True, True])
      #print(df)
      step1_dedup = overlap_deduplications.deduplicate_full_overlaps(df)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("100% deduplication done.")
      #70% Overlap
      step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("70% deduplication done.")
      #Less than 70% overlap
      print(step2_dedup)
      step3_dedup = step2_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.below_70_overlap).reset_index(drop=True)
      print("Less than 70% deduplication done.")
      #return something final
      print(step3_dedup)
      return step3_dedup




In [7]:
dataframes = []
col_names = [
    'target name', 'accession', 'query name', 'accession1', 'hmm len',
    'hmm from', 'hmm to', 'seq len', 'ali from', 'ali to',
    'env from', 'env to', 'E-value', 'score', 'bias', 'shifts',
    'stops', 'pipe', 'description of target', 'extra'
]
for file_path in glob(bathsearch_op_path):
    # Check if the file is not empty by checking its size
    if os.path.getsize(file_path) == 0:
        print(f"Skipping completely empty file: {file_path}")
        continue

    try:
        # Read the table
        bathout = pd.read_table(file_path, sep="\s+", header=None, skiprows=2, skipfooter=8, engine='python')
    except pd.errors.EmptyDataError:
        print(f"No data to parse from file (possibly headers only): {file_path}")
        continue

    # Check if the DataFrame is empty after reading
    if bathout.empty:
        print(f"Skipping empty file with headers but no data: {file_path}")
        continue

    # Rename the columns
    bathout.rename(columns=dict(zip(bathout.columns, col_names)), inplace=True)

    # Append the DataFrame to the list
    dataframes.append(bathout)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

In [8]:
df_neg = filtering_operations.get_specific_strand(combined_df,"-")
neg_deduped = overlap_deduplications.choose_strand_operations(df_neg)
df_pos = filtering_operations.get_specific_strand(combined_df,"+")
pos_deduped = overlap_deduplications.choose_strand_operations(df_pos)
complete_bath_deduplication = pd.concat([pos_deduped,neg_deduped])

<ipython-input-5-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-5-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali to'] = df['ali to'].astype(int)
<ipython-input-5-830c4c50e74a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

Processing negative strand data...
Number of elements in relevant strand:  (51674, 22)
Number of exact duplicates or homologs:  (22528, 22)
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains multiple rows
Group contains mult

<ipython-input-6-46dba7b531c9>:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  deduplicated_remhomologs = duplicates_df1.groupby(['target name',overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column], group_keys=False).apply(overlap_deduplications.handle_group).reset_index(drop=True)


Streaming output truncated to the last 5000 lines.
{'target name': 'NODE_9724_length_2321_cov_2.097529_bin.29', 'accession': '-', 'query name': 'NG_055642.1', 'accession1': '-', 'hmm len': 232, 'hmm from': 3, 'hmm to': 228, 'seq len': 2321, 'ali from': 1924, 'ali to': 1256, 'env from': 1930, 'env to': 1247, 'E-value': 5.1999999999999994e-83, 'score': 292.2, 'bias': 1.8, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 1256, 'ali to flip': 1924}
{'target name': 'NODE_9724_length_2321_cov_2.097529_bin.29', 'accession': '-', 'query name': 'A0QTK2', 'accession1': '-', 'hmm len': 228, 'hmm from': 5, 'hmm to': 223, 'seq len': 2321, 'ali from': 1927, 'ali to': 1256, 'env from': 1936, 'env to': 1244, 'E-value': 4.3e-36, 'score': 140.0, 'bias': 0.0, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 1256, 'ali to flip': 1927}
Begining Comparison:  8
Comparing:
Hit A: {'target name': 'NODE_972

<ipython-input-6-46dba7b531c9>:214: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)


...

<ipython-input-6-46dba7b531c9>:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  deduplicated_remhomologs = duplicates_df1.groupby(['target name',overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column], group_keys=False).apply(overlap_deduplications.handle_group).reset_index(drop=True)


Streaming output truncated to the last 5000 lines.
Overlap1: 89.76109215017065%, Overlap2: 99.62121212121212%
Choosing Hit A based on E-value
{'target name': 'NODE_9717_length_2018_cov_1.981151_bin.54', 'accession': '-', 'query name': 'MF_01987', 'accession1': '-', 'hmm len': 310, 'hmm from': 2, 'hmm to': 309, 'seq len': 2018, 'ali from': 402, 'ali to': 1280, 'env from': 399, 'env to': 1283, 'E-value': 3.8e-93, 'score': 324.5, 'bias': 6.1, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '+'}
{'target name': 'NODE_9717_length_2018_cov_1.981151_bin.54', 'accession': '-', 'query name': 'P83736', 'accession1': '-', 'hmm len': 324, 'hmm from': 40, 'hmm to': 297, 'seq len': 2018, 'ali from': 492, 'ali to': 1247, 'env from': 462, 'env to': 1286, 'E-value': 2.9e-09, 'score': 46.9, 'bias': 0.5, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '+'}
Begining Comparison:  0
Comparing:
Hit A: {'target name': 'NODE_9717_length_2018_cov_1

<ipython-input-6-46dba7b531c9>:233: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)


Overlap calculations done on columns: ali from ali to
70% deduplication done.
                                     target name accession query name  \
0     NODE_10022_length_2530_cov_2.013737_bin.68         -     P77211   
1     NODE_10022_length_2530_cov_2.013737_bin.68         -   MF_00387   
2     NODE_10034_length_3867_cov_1.920776_bin.10         -     Q45585   
3     NODE_10034_length_3867_cov_1.920776_bin.10         -     P0A9U1   
4     NODE_10056_length_2262_cov_1.996375_bin.40         -   MF_00685   
...                                          ...       ...        ...   
2768   NODE_9997_length_2187_cov_1.946529_bin.12         -   MF_00819   
2769   NODE_9997_length_2187_cov_1.946529_bin.12         -   MF_00819   
2770   NODE_9997_length_2187_cov_1.946529_bin.12         -   MF_00015   
2771   NODE_9998_length_2081_cov_2.067127_bin.40         -   MF_01395   
2772   NODE_9998_length_2081_cov_2.067127_bin.40         -   MF_00823   

     accession1  hmm len  hmm from  hmm to  s

<ipython-input-6-46dba7b531c9>:238: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  step3_dedup = step2_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.below_70_overlap).reset_index(drop=True)


In [9]:
complete_bath_deduplication.shape

(6009, 22)

In [13]:
#mark with bin interval
coverage_intervals = {
    "1.4-1.6": (1.4, 1.6),
    "1.9-2.1": (1.9, 2.1),
    "2.4-2.6": (2.4, 2.6),
    "3.8-4.2": (3.8, 4.2),
    "7.5-8.5": (7.5, 8.5)
}


def get_coverage_interval(coverage):
    for interval, (low, high) in coverage_intervals.items():
        if low <= coverage <= high:
            return interval
    return "Outside defined intervals"


def categorize_row(row):
    parts = row['target name'].split('_')
    coverage = float(parts[5][4:])  # Assuming the format is always the same

    # Check for bin status
    if "bin" in row['target name']:
        bin_status = "binned"
    elif "unbinned" in row['target name']:
        bin_status = "unbinned"
    else:
        bin_status = "unknown"

    interval = get_coverage_interval(coverage)

    return pd.Series([interval, bin_status])

complete_bath_deduplication[['coverage interval', 'bin status']] = complete_bath_deduplication.apply(categorize_row, axis=1)


In [14]:
complete_bath_deduplication.to_excel(deduplication_op_path)

In [11]:
@staticmethod
def get_statistics():
    return {
        "Initial Count": overlap_deduplications.initial_count,
        "Count After 100% Deduplication": overlap_deduplications.count_after_100,
        "Count After 70% Deduplication": overlap_deduplications.count_after_70,
        "Count After <70% Deduplication": overlap_deduplications.count_after_less_than_70
    }

stats = get_statistics()
print(stats)

{'Initial Count': 50254, 'Count After 100% Deduplication': 34473, 'Count After 70% Deduplication': 42921, 'Count After <70% Deduplication': 4125}


### Single File Test

In [12]:
bathout = pd.read_table('/content/test_case.tbl', sep="\s+",skiprows =1, header=None)
bathout
# Define column names
col_names = ['target name', 'accession', 'query name', 'accession1', 'hmm len', 'hmm from', 'hmm to',
              'seq len', 'ali from', 'ali to', 'env from', 'env to', 'E-value', 'score', 'bias',
              'shifts', 'stops', 'pipe', 'description of target', 'extra']

# Rename columns
bathout.rename(columns=dict(zip(bathout.columns, col_names)), inplace=True)

# Process negative strand
#df_neg = filtering_operations.get_specific_strand(bathout, "-")
#neg_deduped = overlap_deduplications.choose_strand_operations(df_neg)

# Process positive strand
df_pos = filtering_operations.get_specific_strand(bathout, "+")
pos_deduped = overlap_deduplications.choose_strand_operations(df_pos)


FileNotFoundError: [Errno 2] No such file or directory: '/content/test_case.tbl'

In [ ]:
pos_deduped